In [232]:
#Импортируем необходимые модули  из Pullenti и объявляем переменные
from pullenti_wrapper.langs import (
    set_langs,
    RU
)
set_langs([RU])
from pullenti_wrapper.processor import (
    Processor,
    GEO,
    ADDRESS
)
from pullenti_wrapper.referent import Referent
import pandas as pd
import json

from tqdm import tqdm

In [ ]:

processor = Processor([GEO, ADDRESS])
#Текст, содержащий адреса
text = 'Новокузнецк автотранспортная 45. Новокузнецк Рокоссовского 16. Новокузнецк дачное товарищество Робинзон. Новокузнецк Ильинское шоссе'#'При решении возникшей проблемы я уже было отступила, но оператор связи своими смссообщениями всё таки подвинула меня для окончательного решения возникшей ситуации /Москва, улица Марьинский парк'#Москва, улица Марьинский парк ,д.21,кор.2'#Железногорск, Энтузиастов 10Б"#'Москва, улица Марьинский парк ,д.21,кор.2'
#Функция формирования словаря с адресами
def display_shortcuts(referent, level=0):
    tmp = {}
    a = ""
    b = ""
    for key in referent.__shortcuts__:
        value = getattr(referent, key)
        if value in (None, 0, -1):
            continue
        if isinstance(value, Referent):
            display_shortcuts(value, level + 1)
        else:
            if key == 'type':
                a = value 
            if key == 'name':
                b = value
                # print('ok', value)
            if key == 'house':
                a = "дом"
                b = value
                tmp[a] = b
            if key == 'flat':
                a = "квартира"
                b = value
                # print('ok', value)
                tmp[a] = b
            if key == 'corpus':
                a = "корпус"
                b = value
                tmp[a] = b
    tmp[a] = b
    addr.append(tmp)
    
#Использование функции display_shortcuts и вывод результатов
addr = []
result = processor(text)
#кусочек без защиты
# referent = result.matches[0].referent
# display_shortcuts(referent)
# print(addr)
# st=[]
# i=0
# while i<len(addr):
#     for key,value in addr[i].items():
#         st+=key, value
#     i+=1
# print(' '.join(st))
#кусочек с защитой
try:
    referent = result.matches[0].referent
    display_shortcuts(referent)
    print(addr)
    st=[]
    i=0
    while i<len(addr):
        for key,value in addr[i].items():
            st+=key, value
        i+=1
    print(' '.join(st))
except:
    print('ну чтож, у вас ошибка')

In [ ]:
import pandas as pd
data1 = pd.read_csv(r"C:\Users\vitaly.flerin\Desktop\result.csv" \
    , sep=',' , encoding='cp1251') #, index_col=0 - индекс 
data1.head()  

In [ ]:
import numpy as np
df = data1
df['mark_6']=df['mark_6'].apply(str)
df['mark_6_do'] = np.nan
df

In [ ]:
df['mark_6']

In [ ]:
x = -1
for i, row in df.iterrows(): #перебирает таблицу по строкам
    x = x + 1
    if row['mark_6'].isdigit() : #проверка на число
        continue #вылетаем, если число

    else:
        addr = []
        
        print(x) #печатаю номер строки, чтобы видно, где ошибка
        print(row['mark_6'])
        
        #print(result)
        try: #тут обращение к той функции 
            result = processor(row['mark_6'])
            referent = result.matches[0].referent
            display_shortcuts(referent)
            #print(addr)
            ##addr2 = ', '.join([f'{key.capitalize()}:{value}'for key , value in addr[0].items()])
            #print('!1',addr2)#','.join(addr))
            st=[]
            i=0
            while i<len(addr):
                for key,value in addr[i].items():
                    st+=key, value
                i+=1
            df.loc[x,'mark_6_do'] = ' '.join(st) #запись в столбец
            print(' '.join(st))#','.join(addr))
        except:
            continue

    


In [252]:
df.loc[14411, 'mark_6'] #посмотреть конкретную ячейку

'Новокузнецк автотранспортная 45. Новокузнецк Рокоссовского 16. Новокузнецк дачное товарищество Робинзон. Новокузнецк Ильинское шоссе'

In [268]:
df

,create_date,subs_id,mark_6,mark_6_do
0,2022-09-14,300002341055,У вас есть запись разговоров вот и смотрите ни...,NaN
1,2022-11-03,200102964910,"На все вопросы сотрудники отвечали доступно, п...",NaN
2,2022-07-19,46541013,Ростовская обл цимлянский район ст красноярска...,область РОСТОВСКАЯ
3,2022-04-04,62082420,Спасибо вам тоже,NaN
4,2022-01-31,200073346869,Частный дом в Калужской обл. Перемышльского ро...,область КАЛУЖСКАЯ
...,...,...,...,...
181747,2022-04-18,59625814,Благодарю,NaN
181748,2022-07-21,200052041366,Иногда по всему Ульяновску,город УЛЬЯНОВСК
181749,2022-10-11,200085763712,Посёлок Батецкий,поселок БАТЕЦКИЙ
181750,2022-01-26,51783545,Добринский район Липецкой обл.,область ЛИПЕЦКАЯ район ДОБРИНСКИЙ


In [269]:
df.to_excel(r"C:\Users\vitaly.flerin\Desktop\res_pullenti.xlsx", index=False)